############## HAZY IPA SCRAPE (also know as new-england or juicy IPA) ################
unfortunately there isn't a hazy ipa/ NEIPA/ juicy IPA style category on brewersfriend so 

In [88]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys 

driver = webdriver.Chrome()  
driver.get('https://www.brewersfriend.com/search/')

#close popup
xpath = "//*[@class='fancybox-item fancybox-close']"
WebDriverWait(driver, 35).until(EC.element_to_be_clickable((By.XPATH, xpath))).click()
#time.sleep(35) #Should use webDriverWait here instead - takes awhile for popup to appear
#driver.find_element_by_xpath("//*[@class='fancybox-item fancybox-close']").click()

#find all search bars available
#searches[0] = method,... searches[1,2,3,4,5,6] = [units,style,fermentables,hops,yeasts,other]
time.sleep(1)
searches = driver.find_elements_by_xpath("//*[@class='search']")

#input brew method
searches[0].clear()                #clear any populated text (there isn't any in this case but its good practice)
searches[0].send_keys('All Grain') #input all grain into searchbar
searches[0].send_keys(Keys.ENTER)  #hit enter

#input brew style
searches[2].clear()
searches[2].send_keys('Specialty IPA: New England IPA')
searches[2].send_keys(Keys.ENTER)

#get total pages of recipes
time.sleep(2) #use webDriverWait eventually
totalPages = int(driver.find_element_by_xpath("//*[contains(text(), 'Last')]").get_attribute('data-page-number'))

recipeLinks = [] # initialize list to store recipeLinks

#iterate through all of the pages and get all recipe links on each page
currentPage = 0 #initialize current page before entering while loop
#emulate do-while loop
while True:
    currentPage = currentPage + 1
    time.sleep(2) #give it time after clicking next page to avoid stale element error
    recipes = driver.find_elements_by_xpath("//*[@class='recipetitle']")

    #get links for each recipe
    for i in range(0,(len(recipes))):
        recipeLinks.append(recipes[i].get_attribute("href"))
     
    #check to see if on last page and break from while if so, 
    #otherwise, go to next page
    if (currentPage == totalPages):
        break
    
    #go to next page
    time.sleep(random.random()*5+1)
    #click the last available next (avoid clicking links with recipe names that have 'next' in them)
    nexts = driver.find_elements_by_xpath("//*[contains(text(), 'Next')]") 
    nexts[len(nexts)-1].click() 


In [91]:
#print number of recipes and save recipe links to csv
print("Number of recipes: ", len(recipeLinks))
df = pd.DataFrame(recipeLinks, columns = ['NEIPA links'])
df.head()
#df.to_csv('NEIPAlinks.csv', encoding = 'utf-8', index = False)


Number of recipes:  2854


,NEIPA links
0,https://www.brewersfriend.com/homebrew/recipe/...
1,https://www.brewersfriend.com/homebrew/recipe/...
2,https://www.brewersfriend.com/homebrew/recipe/...
3,https://www.brewersfriend.com/homebrew/recipe/...
4,https://www.brewersfriend.com/homebrew/recipe/...


In [ ]:
#now that we have the links scraped, we can start 
#scraping ingredients and other pertinent info
import time
import pandas as pd
import random
import csv
import urllib.request
import requests
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from itertools import cycle
import traceback
import matplotlib.pyplot as plt
import numpy as np
import re

start_time = time.time()

#initialize all parameters
#awardWin = [] #binary yes or no if its an award winner
recipeURL = []
name = []
author = [] #creator of recipe

#beer stats
method = [] #brew method (all grain, extract, etc)
style = [] #brew style (NEIPA, cream ale, etc)
boilTime = []
batchSize = [] #fermentor volume
preBoilSize = []
postBoilSize = []
preBoilGravity = [] #recipe based estimate
efficiency = []
source = []
noChill = [] #letting it cool by itself with no external help
rating = []
numRating = [] #number of people who rated beer
hopUtil = [] #hop utilization
calories = []
carbs = []
views = [] #number of views recipe has
brews = [] #number of other users that have brewed the beer
links = [] #url might be present, for example as proof for award winners
dateCreated = [] #date recipe was created

og = [] #original gravity
fg = [] #final gravity
abv = [] #alcohol by volume
ibu = [] #international bitterness units
srm = [] #standard reference method (beer color)
mashPh = [] #ph of mash

fermNames = []
fermAmounts = []
fermPPGs    = [] #points per pound per gallon
fermLs      = [] #°L (degrees Lovibond)
fermBillPs  = [] #bill percentage

hopAmounts = []
hopNames = []
hopForms = [] #i.e., leaf/whole or pellet
hopAAs = [] #alpha acid content
hopUses = [] #when added i.e., during boil, dry hop, etc
hopTimes = [] #when to add hops
hopIBUs = [] #international bitterness units
hopBillPs = [] #bill percentage

yeastNames = []
yeastAmounts = []
yeastAttenuations = [] #yeast attenuation (avg)
yeastOptTemps = [] #optimum temp
yeastFlocs = [] #flocculation
yeastStarters = [] #binary yes or no
yeastFermTemps = [] #fermentation temp
yeastPRs = [] #pitch rate

#target water profile
ca = [] #Ca(+2)
mg = [] #Mg(+2)
na = [] #Na(+)
cl = [] #Cl(-)
so4 = [] #SO4(-2)
hco3 = [] #HCO3(-)

priming = [] #priming info (how to carbonate)

otherNames = []
otherAmounts = []
otherTypes = []
otherUses = []
otherTimes = []

notes = [] #any additional notes author provided

data = pd.read_csv('NEIPAlinks 1-26-20.csv', encoding = "utf-8")
count = 0
#loop through each recipe
for link in data['NEIPA links'][0:len(data['NEIPA links'])]:#len(data['NEIPA links'])
    count = count+1
    
    #loop through each recipe
    url = link
    recipeURL.append(url)
    hdr = {'User-Agent': 'Mozilla/5.0'} # used to avoid 403 error
    req = Request(url, headers=hdr)
    try:
        page = urlopen(req)
    except HTTPError as e:
        print('The server couldn\'t fulfill the request.')
        print('Error code: ', e.code)
        print(link)
        sys.exit(1)
    except URLError as e:
        print('We failed to reach a server.')
        print('Reason: ', e.reason)
        print(link)
        sys.exit(1)
    else:
        # everything is fine
        soup = BeautifulSoup(page, "lxml")
        
        #get name of recipe
        name_container = soup.find('h3', itemprop='name')
        if name_container is None:
            name.append('NA')
        else:
            name.append(name_container.text.strip())

        #get author of recipe
        author_container = soup.find('span', itemprop = "author")
        if author_container is None:
            author.append('NA')
        else:
            author.append(author_container.text.strip())

        #get date created
        date_container = soup.find('div', class_="extra") 
        if date_container is None:
            dateCreated.append('NA')
        else:
            dateCreated.append(date_container.text.strip().replace('Created', ''))

        #get beer stats
        beerStats_container = soup.find('div', class_ = "description")
        if beerStats_container is None:
            method.append('NA')
            style.append('NA')
            boilTime.append('NA')
            batchSize.append('NA')
            preBoilSize.append('NA')
            preBoilGravity.append('NA')
            efficiency.append('NA')
            source.append('NA')
            noChill.append('NA')
            rating.append('NA')
            numRating.append('NA')
            hopUtil.append('NA')
            calories.append('NA')
            carbs.append('NA')
            views.append('NA')
            brews.append('NA')
            links.append('NA')
            dateCreated.append("NA")
        else:
            for i in range(0,len(beerStats_container.find_all('span', class_='viewStats'))):
                if beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Method:':
                    method.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Style:':
                    style.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Boil Time:':
                    boilTime.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Batch Size:':
                    batchSize.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Pre Boil Size:':
                    preBoilSize.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Post Boil Size:':
                    postBoilSize.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Pre Boil Gravity:':
                    preBoilGravity.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Efficiency:':
                    efficiency.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Source:':
                    source.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Rating:':
                    rating.append(beerStats_container.find_all('span', class_='viewStats')[i].find('span',itemprop = "ratingValue").text.strip())
                    numRating.append(beerStats_container.find_all('span', class_='viewStats')[i].find('span',itemprop = "reviewCount").text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'No Chill:':
                    noChill.append(' '.join(beerStats_container.find_all('span', class_='viewStats')[i].text.split()).split('No Chill: ',1)[1])
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Calories:':
                    calories.append(beerStats_container.find_all('span', class_='viewStats')[i].text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Hop Utilization:':
                    hopUtil.append(beerStats_container.find_all('span', class_='viewStats')[i].find('strong').text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'Carbs:':
                    carbs.append(beerStats_container.find_all('span', class_='viewStats')[i].text.strip())
                elif beerStats_container.find_all('span', class_='viewStats')[i].find('span', class_='firstLabel').text == 'URL:':
                    links.append(beerStats_container.find_all('span', class_='viewStats')[i].find('a').text.strip())
                else:
                    print("Theres another general stat you're not aware of: ", url)

            #assign NA to any missing stats by making a list of whats present and checking if any are missing
            statNames = [] 
            for i in range(0, len(beerStats_container.find_all('span', class_='viewStats'))):
                statNames.append(beerStats_container.find_all('span', class_='viewStats')[i].find('span',class_="firstLabel").text)

            if 'Method:' not in statNames:
                method.append('NA')
            if 'Style:' not in statNames:
                style.append('NA')
            if 'Boil Time:' not in statNames:
                boilTime.append('NA')
            if 'Batch Size:' not in statNames:
                batchSize.append('NA')
            if 'Pre Boil Size:' not in statNames:
                preBoilSize.append('NA')
            if 'Post Boil Size:' not in statNames:
                postBoilSize.append('NA')
            if 'Pre Boil Gravity:' not in statNames:
                preBoilGravity.append('NA')
            if 'Efficiency:' not in statNames:
                efficiency.append('NA')
            if 'Source:' not in statNames:
                source.append('NA')
            if 'Rating:' not in statNames:
                rating.append('NA')
                numRating.append('NA')
            if 'Hop Utilization:' not in statNames:
                hopUtil.append('NA')
            if 'No Chill:' not in statNames:
                noChill.append('NA')
            if 'Calories:' not in statNames:
                calories.append('NA')
            if 'Carbs:' not in statNames:
                carbs.append('NA')
            if 'URL:' not in statNames:
                links.append("NA")

        #get remaining stats in bar below main stats
        beerStatsOther_container = soup.find('div', class_='appbounds minstats')
        if beerStatsOther_container is None:
            og.append('NA')
            fg.append('NA')
            abv.append('NA')
            ibu.append('NA')
            srm.append('NA')
            mashPh.append('NA')
        else: 
            og.append(beerStatsOther_container.find('div', class_='value ogBatch').text.strip())
            fg.append(beerStatsOther_container.find('div', class_='value fgBatch').text.strip())
            abv.append(beerStatsOther_container.find('div', class_='value abvMin').text.strip())
            ibu.append(beerStatsOther_container.find('div', class_='value ibuMin').text.strip())
            srm.append(beerStatsOther_container.find('div', class_='value srmMin').text.strip())
            mashPh.append(beerStatsOther_container.find('div', class_='value phMin').text.strip())

        # get fermentable ingredient list
        fermAmount = []
        fermName   = [] 
        fermPPG    = [] #points per pound per gallon
        fermL      = [] #°L (degrees Lovibond)
        fermBillP  = [] #bill percentage

        ferm_container = soup.find('div', class_ = "brewpart", id = 'fermentables') #location of fermentables
        if ferm_container is None: #if fermentables aren't available, record 'NA'
            fermAmount.append('NA')
            fermName.append('NA')
            fermPPG.append('NA')
            fermL.append('NA')
            fermBillP.append('NA')
        else:          #else access specific partsof table where data are recorded, # i = table row, j = table column 
            for i in range(1,len(ferm_container.table.find_all('tr'))-1): #-1 to avoid last row (total weight)  
                for j in range(0,len(ferm_container.table.find_all('tr')[i])):
                    if j == 0: #amounts are in 1st column
                        fermAmount.append(ferm_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                    elif j == 1: #names in 2nd column...etc
                        #some recipes have the grain names hyperlinked, if hyperlinked there will be two
                        #text fields: the hop name and amount of hops
                        #e.g., this allows it to scrape just 'American - Wheat' instead of 'American - Wheat1.4 lb Wheat'
                        if (ferm_container.table.find_all('tr')[i].find_all('td')[j].find('a') is None):
                            fermName.append(ferm_container.table.find_all('tr')[i].find_all('td')[j].find('span').text.strip())
                        else:
                            fermName.append(ferm_container.table.find_all('tr')[i].find_all('td')[j].find('a').text.strip())
                    elif j == 2: #ppg
                        fermPPG.append(ferm_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                    elif j == 3: #°L (degrees Lovibond)
                        fermL.append(ferm_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                    elif j == 4: #bill percentage
                        fermBillP.append(ferm_container.table.find_all('tr')[i].find_all('td')[j].text.strip())   

        fermAmounts.append(fermAmount)
        fermNames.append(fermName)
        fermPPGs.append(fermPPG)
        fermLs.append(fermL)
        fermBillPs.append(fermBillP)

        # get hop lists
        hopAmount = []
        hopName = []
        hopForm = [] #i.e., leaf/whole or pellet
        hopAA = [] #alpha acid content
        hopUse = [] #when added i.e., during boil, dry hop, etc
        hopTime = []
        hopIBU = [] #international bitterness units
        hopBillP = [] #bill percentage

        hop_container = soup.find('div', class_ = "brewpart", id = 'hops')
        if hop_container is None: #if hop table doesnt exist, record 'NA'
            hopAmount.append('NA')
            hopName.append('NA')
            hopForm.append('NA')
            hopAA.append('NA')
            hopUse.append('NA')
            hopTime.append('NA')
            hopIBU.append('NA')
            hopBillP.append('NA')
        else:                     #else record hopnames and temp/times
            if hop_container.find('div', id='hopsSummary', style='display: none;') is not None: #if hops table isn't in summary view
                for i in range(1,len(hop_container.table.find_all('tr'))):  # i = table row, j = table column 
                    for j in range(0,len(hop_container.table.find_all('tr')[i])):
                        if j == 0: #amounts are in 1st column
                            hopAmount.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 1: #names in 2nd column...etc
                            #some recipes have the hop names hyperlinked, if hyperlinked there will be two
                            #text fields: the hop name and amount of hops
                            #e.g., this allows it to scrape just 'Citra' instead of 'Citra1 oz Citra Hops'
                            if hop_container.table.find_all('tr')[i].find_all('td')[j].find('a') is None:
                                hopName.append(hop_container.table.find_all('tr')[i].find_all('td')[j].find('span').text.strip())
                            else:
                                hopName.append(hop_container.table.find_all('tr')[i].find_all('td')[j].find('a').text.strip())
                        elif j == 2: #form
                            hopForm.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 3: #AA (degrees Lovibond)
                            hopAA.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 4: #Use
                            hopUse.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 5: #Time
                            hopTime.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 6: #IBU
                            hopIBU.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 7: #bill percentage
                            hopBillP.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())   
            else: #else hops is in summary view - smaller table
                for i in range(1,len(hop_container.table.find_all('tr'))-1):  #-1 since last row is total (dont need)
                    for j in range(0,len(hop_container.table.find_all('tr')[i])): # i = table row, j = table column 
                        if j == 0: #amounts are in 1st column
                            hopAmount.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 1: 
                            #some recipes have the hop names hyperlinked, if hyperlinked there will be two
                            #text fields: the hop name and amount of hops
                            #e.g., this allows it to scrape just 'Citra' instead of 'Citra1 oz Citra Hops'
                            if hop_container.table.find_all('tr')[i].find_all('td')[j].find('a') is None:
                                #names and form are in 2nd column when hops are in summary view
                                unparsed = hop_container.table.find_all('tr')[i].find_all('td')[j].find('span').text.strip()
                                name_ = unparsed.split('(')[0].strip() #name_ has underscore because name is list of recipe names
                                form = unparsed.split('(')[1].replace(')','').strip()
                                hopName.append(name_)
                                hopForm.append(form)
                            else:
                                #names and form are in 2nd column when hops are in summary view
                                unparsed = hop_container.table.find_all('tr')[i].find_all('td')[j].find('a').text.strip()
                                name_ = unparsed.split('(')[0].strip()
                                form = unparsed.split('(')[1].replace(')','').strip()
                                hopName.append(name_)
                                hopForm.append(form)
                        elif j == 2: #IBU
                            hopIBU.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                        elif j == 3: #bill percentage
                            hopBillP.append(hop_container.table.find_all('tr')[i].find_all('td')[j].text.strip()) 

                        #hop AA, use, and time aren't available in summarized table
                        hopAA.append('NA') #we can look up average AA content and add it later
                        hopUse.append('NA')
                        hopTime.append('NA')
                
                
        hopAmounts.append(hopAmount)
        hopNames.append(hopName)
        hopForms.append(hopForm)
        hopAAs.append(hopAA)
        hopUses.append(hopUse)
        hopTimes.append(hopTime)
        hopIBUs.append(hopIBU)
        hopBillPs.append(hopBillP)


        #get yeast info
        yeastName = []
        yeastAmount = []
        yeastAttenuation = [] #yeast attenuation (avg)
        yeastOptTemp = [] #optimum temp
        yeastFloc = [] #flocculation
        yeastStarter = [] #binary yes or no
        yeastFermTemp = [] #fermentation temp
        yeastPR = [] #pitch rate

        #get yeast  
        #need to fix so it can scrape multiple yeasts - cant find a recipe with 2 yeasts atm tho
        yeast_container = soup.find('div', class_ = "brewpart", id = 'yeasts')
        if yeast_container is None:
            yeastName.append('NA')
            yeastAmount.append('NA')
            yeastAttenuation.append('NA')
            yeastOptTemp.append('NA')
            yeastFloc.append('NA')
            yeastStarter.append('NA')
            yeastFermTemp.append('NA')
            yeastPR.append('NA')
        else:
            for i in range(0,len(yeast_container.find_all('tbody'))): #most recipes use 1 yeast but some have 2+
                yeastName.append(yeast_container.find_all('thead')[i].find('span').text.strip())
                #table is in a weird format compared to the others, also varies in dimensions
                #so we'll take in the entire text of the table and parse it
                unparsed = yeast_container.find_all('tbody')[i].find_all('tr')[0].find_all('td')[0].text
                #replace any whitespace with just 1 space
                unparsed = ' '.join(unparsed.split())
                #make it comma delimited so we can make a list of features
                unparsed = unparsed.replace('Attenuation', ', Attenuation').replace('Flocculation', ', Flocculation')
                unparsed = unparsed.replace('Optimum', ', Optimum').replace('Starter', ', Starter')
                unparsed = unparsed.replace('Fermentation', ', Fermentation').replace('Pitch', ', Pitch')
                unparsed = unparsed.split(',')

                #depending on what feature each element of the list is, assign respective value
                for i in range(0,len(unparsed)):
                    if 'Amount:' in unparsed[i]:
                        yeastAmount.append(unparsed[i].split('Amount:')[1].strip())
                    elif 'Attenuation (avg):' in unparsed[i]:
                        yeastAttenuation.append(unparsed[i].split('Attenuation (avg):')[1].strip()+' (average)')
                    elif 'Attenuation (custom):' in unparsed[i]:
                        yeastAttenuation.append(unparsed[i].split('Attenuation (custom):')[1].strip()+' (custom)')
                    elif 'Flocculation:' in unparsed[i]:
                        yeastFloc.append(unparsed[i].split('Flocculation:')[1].strip())
                    elif 'Optimum Temp:' in unparsed[i]:
                        yeastOptTemp.append(unparsed[i].split('Optimum Temp:')[1].strip())
                    elif 'Starter:' in unparsed[i]:
                        yeastStarter.append(unparsed[i].split('Starter:')[1].strip())
                    elif 'Fermentation Temp:' in unparsed[i]:
                        yeastFermTemp.append(unparsed[i].split('Fermentation Temp:')[1].strip())
                    elif 'Pitch Rate:' in unparsed[i]:
                        yeastPR.append(unparsed[i].split('Pitch Rate:')[1].strip())
                    else:
                        print("Theres a yeast feature you're unaware of: ", link)

                #insert 'NA' if features aren't available
                if not any('Amount' in x for x in unparsed):
                    yeastAmount.append('NA')
                elif not any('Attenuation' in x for x in unparsed):
                    yeastAttenuation.append('NA')
                elif not any('Flocculation' in x for x in unparsed):
                    yeastFloc.append('NA')
                elif not any('Optimum Temp' in x for x in unparsed):
                    yeastOptTemp.append('NA')
                elif not any('Starter' in x for x in unparsed):
                    yeastStarter.append('NA')
                elif not any('Fermentation Temp' in x for x in unparsed):
                    yeastFermTemp.append('NA')
                elif not any('Pitch Rate' in x for x in unparsed):
                    yeastPR.append('NA')
        
        yeastNames.append(yeastName)
        yeastAmounts.append(yeastAmount)
        yeastAttenuations.append(yeastAttenuation)
        yeastFlocs.append(yeastFloc)
        yeastOptTemps.append(yeastOptTemp)
        yeastStarters.append(yeastStarter)
        yeastFermTemps.append(yeastFermTemp)
        yeastPRs.append(yeastPR)

        #get priming info
        priming_container = soup.find('div', class_='brewpart',  id = "primingmethod")
        if priming_container is None:
            priming.append('NA')
        else:
            priming.append(priming_container.find('td').text.strip().replace('\xa0', '').replace('\t','').replace('\n', ','))

        #get other ingredients
        otherAmount = []
        otherName = []
        otherType = []
        otherUse = []
        otherTime = []

        other_container = soup.find('div', class_ = "brewpart", id = 'others')
        if other_container is None:
            otherAmount.append('NA')
            otherName.append('NA')
            otherType.append('NA')
            otherUse.append('NA')
            otherTime.append('NA')
        else:
            for i in range(1,len(other_container.table.find_all('tr'))):  # i = table row, j = table column 
                for j in range(0,len(other_container.table.find_all('tr')[i])):
                    if j == 0: #amounts are in 1st column
                        otherAmount.append(other_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                    elif j == 1: #names in 2nd column...etc
                        #similar problem here as with hopNames and fermNames, see them for more details
                        if other_container.table.find_all('tr')[i].find_all('td')[j].find('a') is None:
                            otherName.append(other_container.table.find_all('tr')[i].find_all('td')[j].find('span').text.strip())
                        else:
                            otherName.append(other_container.table.find_all('tr')[i].find_all('td')[j].find('a').text.strip())
                    elif j == 2: #type
                        otherType.append(other_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                    elif j == 3: #use
                        otherUse.append(other_container.table.find_all('tr')[i].find_all('td')[j].text.strip())
                    elif j == 4: #time
                        otherTime.append(other_container.table.find_all('tr')[i].find_all('td')[j].text.strip())

        otherNames.append(otherName)
        otherAmounts.append(otherAmount)
        otherTypes.append(otherType)
        otherUses.append(otherUse)
        otherTimes.append(otherTime)

        #get target water profile
        water_container = soup.find('div',class_='brewpart',id='water')
        if water_container is None:
            ca.append('NA')
            mg.append('NA')
            na.append('NA')
            cl.append('NA')
            so4.append('NA')
            hco3.append("NA")
        else:
            ca.append(water_container.find('table').find_all('td')[0].text.strip())
            mg.append(water_container.find('table').find_all('td')[1].text.strip())
            na.append(water_container.find('table').find_all('td')[2].text.strip())
            cl.append(water_container.find('table').find_all('td')[3].text.strip())
            so4.append(water_container.find('table').find_all('td')[4].text.strip())
            hco3.append(water_container.find('table').find_all('td')[5].text.strip())


        #get notes 
        notes_container = soup.find('div',class_='ui message')
        if notes_container is None:
            notes.append('NA')
        else:
            notes.append(notes_container.text.strip())

        #get number of views recipe has and number of times other users have brewed recipe
        viewsAndBrews_container = soup.find('div', class_='ui two statistics')
        if viewsAndBrews_container is None:
            views.append('NA')
            brews.append('NA')
        else:
            views.append(viewsAndBrews_container.find_all('div',class_='value')[0].text.strip())
            brews.append(viewsAndBrews_container.find_all('div',class_='value')[1].text.strip())

        #save file every 250 recipes
        if (count % 250) == 0:
            df = pd.DataFrame({'Recipe URL':recipeURL, "Name":name, "Author":author, 'Date Created':dateCreated,
                               'URL Provided':links,'Views':views,'Brews':brews,'OG':og, 'FG':fg, 'ABV':abv,
                               'IBU':ibu,'SRM':srm,'Mash pH':mashPh,"Method":method, 'Style':style,
                               'Boil Time':boilTime, 'Batch Size': batchSize,'Pre Boil Size':preBoilSize, 
                               'Post Boil Size': postBoilSize, 'Pre Boil Gravity':preBoilGravity, 'Efficency':efficiency,
                               'Source':source,'Rating':rating,'Rating Count':numRating, 'No Chill':noChill, 
                               'Hop Utilization':hopUtil,'Calories':calories, 'Carbs':carbs,'Grain Bill':fermNames, 
                               'Grain Amounts': fermAmounts,'Grain PPGs': fermPPGs,'Grain Degree Lovibond': fermLs, 
                               'Grain Bill Percentage':fermBillPs,'Hop Names': hopNames, 'Hop Amounts': hopAmounts, 
                               'Hop Forms': hopForms, 'AA content': hopAAs,'Hop Uses':hopUses, 'Hop Times': hopTimes,
                               'Hop IBUs': hopIBUs, 'Hop Bill Percentage':hopBillPs,'OtherIngredients':otherNames, 
                               'OtherAmounts':otherAmounts,'OtherTypes':otherTypes,'OtherUses':otherUses,
                               'OtherTimes':otherTimes, 'Ca':ca,'Mg':mg,'Na':na,'Cl':cl,'SO4':so4,'HCO3':hco3,
                               'Priming Info':priming,'Yeast Name':yeastNames, 'Yeast Amount':yeastAmounts,
                               'Yeast Attenuation': yeastAttenuations, 'Yeast Opt Temp':yeastOptTemps,
                               'Yeast Flocculation':yeastFlocs,'Yeast Starter':yeastStarters,
                               'Yeast Ferm Temp':yeastFermTemps,'Yeast Pitch Rate':yeastPRs,'Notes':notes})
        
            df.to_csv('NEIPA'+str(count)+'.csv', encoding = 'utf-8', index = False)
            
        
        # pause loop to avoid overloading server
        #time.sleep(random.randint(5,9))
        time.sleep(random.randint(3,6))

print("--- %s seconds ---" % (time.time() - start_time))
#takes 13 seconds to scrape 5 pages <==> 2.6s to scrape each page
    


In [11]:

dfData = pd.DataFrame({'Recipe URL':recipeURL, "Name":name, "Author":author, 'Date Created':dateCreated,
                   'URL Provided':links,'Views':views,'Brews':brews,'OG':og, 'FG':fg, 'ABV':abv,
                   'IBU':ibu,'SRM':srm,'Mash pH':mashPh,"Method":method, 'Style':style,
                   'Boil Time':boilTime, 'Batch Size': batchSize,'Pre Boil Size':preBoilSize, 
                   'Post Boil Size': postBoilSize, 'Pre Boil Gravity':preBoilGravity, 'Efficency':efficiency,
                   'Source':source,'Rating':rating,'Rating Count':numRating, 'No Chill':noChill, 
                   'Hop Utilization':hopUtil,'Calories':calories, 'Carbs':carbs,'Grain Bill':fermNames, 
                   'Grain Amounts': fermAmounts,'Grain PPGs': fermPPGs,'Grain Degree Lovibond': fermLs, 
                   'Grain Bill Percentage':fermBillPs,'Hop Names': hopNames, 'Hop Amounts': hopAmounts, 
                   'Hop Forms': hopForms, 'AA content': hopAAs,'Hop Uses':hopUses, 'Hop Times': hopTimes,
                   'Hop IBUs': hopIBUs, 'Hop Bill Percentage':hopBillPs,'OtherIngredients':otherNames, 
                   'OtherAmounts':otherAmounts,'OtherTypes':otherTypes,'OtherUses':otherUses,
                   'OtherTimes':otherTimes, 'Ca':ca,'Mg':mg,'Na':na,'Cl':cl,'SO4':so4,'HCO3':hco3,
                   'Priming Info':priming,'Yeast Name':yeastNames, 'Yeast Amount':yeastAmounts,
                   'Yeast Attenuation': yeastAttenuations, 'Yeast Opt Temp':yeastOptTemps,
                   'Yeast Flocculation':yeastFlocs,'Yeast Starter':yeastStarters,
                   'Yeast Ferm Temp':yeastFermTemps,'Yeast Pitch Rate':yeastPRs,'Notes':notes})

dfData.to_csv('NEIPA'+str(count)+'.csv', encoding = 'utf-8', index = False)
df.head()

,Recipe URL,Name,Author,Date Created,URL Provided,Views,Brews,OG,FG,ABV,...,Priming Info,Yeast Name,Yeast Amount,Yeast Attenuation,Yeast Opt Temp,Yeast Flocculation,Yeast Starter,Yeast Ferm Temp,Yeast Pitch Rate,Notes
0,https://www.brewersfriend.com/homebrew/recipe/...,Avg. Perfect Northeast IPA (NEIPA),kcq101,Tuesday May 31st 2016,NA,"243,386",296,1.062,1.013,6.5%,...,"Method: Keg - Force Carb.,CO2 Level: 2.4 Volumes",[Wyeast - London Ale III 1318],[1],"[78% (custom), NA]",[64 - 74 °F],[High],[No],[],[],Fermentation Dry Hop @ 70% attenuation (1.027)...
1,https://www.brewersfriend.com/homebrew/recipe/...,Bonnie and Clyde Mango Milkshake NEIPA. We use...,Vallka,Friday September 15th 2017,NA,"5,460",7,1.063,1.020,5.65%,...,CO2 Level: 2.35 Volumes,[Wyeast - London Ale III 1318],[1],"[73% (custom), NA]",[18 - 23 °C],[High],[No],[20 °C],[0.35 (M cells / ml / ° P) 124 B cells required],John's ale yeast
2,https://www.brewersfriend.com/homebrew/recipe/...,#SerNEpper,Mazsi,Thursday October 26th 2017,NA,"4,263",0,1.062,1.014,6.3%,...,NA,[The Yeast Bay - Vermont Ale],[1],[78.5% (average)],[18 - 22 °C],[Med/Low],[No],[21 °C],[0.75 (M cells / ml / ° P) 422 B cells required],élesztő:\nWyeast - London Ale III\nWLP Burling...
3,https://www.brewersfriend.com/homebrew/recipe/...,Anniversary N. E. IPA,Ostrander,Saturday September 29th 2012,NA,"3,894",1,1.066,1.018,6.26%,...,"Method: Corn Sugar,Amount: 1.4 oz",[Wyeast - London Ale III 1318],[1],[73% (average)],[64 - 74 °F],[High],[No],[64 °F],[0.75 (M cells / ml / ° P) 298 B cells required],Invert sugar syrup added to primary fermentor ...
4,https://www.brewersfriend.com/homebrew/recipe/...,DIPA v.1,Beatsontoast,Monday September 30th 2013,NA,"3,793",0,1.068,1.015,7.33%,...,CO2 Level: 2.25 Volumes,[Fermentis - Safale - American Ale Yeast US-05...,"[1, 1]","[81% (average), 76.5% (average)]","[12 - 25 °C, 20 - 23 °C]","[Medium, Medium]","[No, No]","[20 °C, 20 °C]",[0.5 (M cells / ml / ° P) 199 B cells required...,NA


In [3]:
dfStats1 = pd.DataFrame({'Recipe URL':recipeURL, "Name":name, "Author":author, 'Date Created':dateCreated,
                         'URL Provided':links,'Views':views,
                         'Brews':brews,'OG':og, 'FG':fg, 'ABV':abv,'IBU':ibu,'SRM':srm,'Mash pH':mashPh})

dfStats2 = pd.DataFrame({"Method":method, 'Style':style,
                        'Boil Time':boilTime, 'Batch Size': batchSize,'Pre Boil Size':preBoilSize, 
                        'Post Boil Size': postBoilSize, 'Pre Boil Gravity':preBoilGravity, 'Efficency':efficiency,
                        'Source':source,'Rating':rating,'Rating Count':numRating, 'No Chill':noChill, 
                        'Hop Utilization':hopUtil,'Calories':calories, 'Carbs':carbs})
print(dfStats1.head())
print(dfStats2.head())

                                          Recipe URL  \
0  https://www.brewersfriend.com/homebrew/recipe/...   
1  https://www.brewersfriend.com/homebrew/recipe/...   

                                                Name  Author  \
0                 Avg. Perfect Northeast IPA (NEIPA)  kcq101   
1  Bonnie and Clyde Mango Milkshake NEIPA. We use...  Vallka   

                  Date Created URL Provided    Views Brews     OG     FG  \
0        Tuesday May 31st 2016           NA  243,340   296  1.062  1.013   
1   Friday September 15th 2017           NA    5,458     7  1.063  1.020   

     ABV     IBU   SRM Mash pH  
0   6.5%   59.26   5.2    5.49  
1  5.65%  101.61  9.45    5.71  
      Method                           Style Boil Time    Batch Size  \
0  All Grain  Specialty IPA: New England IPA    60 min  5.75 gallons   
1  All Grain  Specialty IPA: New England IPA    60 min     23 liters   

  Pre Boil Size Post Boil Size Pre Boil Gravity Efficency  \
0   7.5 gallons             NA  

In [290]:
dfGrainHops = pd.DataFrame({'Grain Bill':fermNames, 'Grain Amounts': fermAmounts,'Grain PPGs': fermPPGs, 
                            'Grain Degree Lovibond': fermLs, 'Grain Bill Percentage':fermBillPs,
                            'Hop Names': hopNames, 'Hop Amounts': hopAmounts, 'Hop Forms': hopForms, 'AA content': hopAAs,
                            'Hop Uses':hopUses, 'Hop Times': hopTimes,'Hop IBUs': hopIBUs, 'Hop Bill Percentage':hopBillPs})
dfRest = pd.DataFrame({'OtherIngredients':otherNames, 'OtherAmounts':otherAmounts,'OtherTypes':otherTypes,
                       'OtherUses':otherUses,'OtherTimes':otherTimes, 'Ca':ca,'Mg':mg,'Na':na,'Cl':cl,'SO4':so4,'HCO3':hco3,
                       'Priming Info':priming,'Yeast Name':yeastNames, 'Yeast Amount':yeastAmounts,
                       'Yeast Attenuation': yeastAttenuations, 'Yeast Opt Temp':yeastOptTemps,'Yeast Flocculation':yeastFlocs,
                       'Yeast Starter':yeastStarters,'Yeast Ferm Temp':yeastFermTemps,'Yeast Pitch Rate':yeastPRs,'Notes':notes})

print(dfGrainHops.head())
print(dfRest.head())


                                          Grain Bill  \
0  [American - Pale 2-Row, American - Wheat, Flak...   
1  [Canadian - Pale 2-Row, American - White Wheat...   
2  [German - Pale Ale, German - Wheat Malt, Flake...   
3  [American - Pale 2-Row, Flaked Wheat, American...   
4  [Crisp - Finest Maris Otter, United Kingdom - ...   

                                       Grain Amounts  \
0                [10.75 lb, 1.4 lb, 1.4 lb, 0.42 lb]   
1  [3.2 kg, 0.9 kg, 0.1 kg, 0.45 kg, 0.28 kg, 2 k...   
2   [7 kg, 1.5 kg, 1.2 kg, 0.25 kg, 0.15 kg, 0.2 kg]   
3                        [14 lb, 1 lb, 1 lb, 0.5 lb]   
4                       [7 kg, 1 kg, 0.2 kg, 0.2 kg]   

                           Grain PPGs              Grain Degree Lovibond  \
0                    [37, 38, 33, 37]                [1.8, 1.8, 2.2, 25]   
1  [36, 40, 37, 33, 41, 4.95, 35, 36]  [1.75, 2.8, 25, 2.2, 1, 0, 2, 10]   
2             [39, 37, 33, 27, 34, 0]          [2.3, 2, 2.2, 3.4, 57, 0]   
3                    [

In [37]:
df = pd.concat([dfStats1,dfStats2,dfGrainHops,dfRest], axis=1)
df.head()
df.to_csv('NEIPAdata', encoding = 'utf-8', index = False)


In [40]:
data['NEIPA links'][402]

'https://www.brewersfriend.com/homebrew/recipe/view/788906/grainfather-caribou-haze'